# Trabajo Práctico 2 - Organización de Datos
## Competencia de Machine Learning
### Facultad de Ingeniería de la Universidad de Buenos Aires
### 95-58: Organización de Datos - 2do Cuat. 2018

#### Integrantes: Gonzalo Diz,  Ariel Windey, Gabriel Robles y Matías El Dócil




#### Objetivo
Determinar, para cada usuario presentado, cuál es la probabilidad de que ese
usuario realice una conversión en Trocafone en un periodo determinado.

#### Fuentes
El archivo "events_up_to_01062018.csv" contiene en el mismo formato utilizado en el TP1
información de eventos realizado en la plataforma para un conjunto de usuarios hasta el
31/05/2018.

Por otro lado el archivo "labels_training_set.csv" indica para un subconjunto de los
usuarios incluidos en el set de eventos "events_up_to_01062018.csv" si los mismos
realizaron una conversión (columna label = 1) o no (columna label = 0) desde el 01/06/2018
hasta el 15/06/2018.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as skl

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
# Carga del set de datos de eventos
eventos = pd.read_csv('../dataset/events_up_to_01062018.csv', low_memory=False)
# Carga del set de datos de labels
labels = pd.read_csv('../dataset/labels_training_set.csv', low_memory=False)


In [3]:
eventos.shape

(2341681, 23)

In [4]:
eventos.event.value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [5]:
eventos.sample(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
1039888,2018-04-20 02:34:01,ad campaign hit,811f78a5,/comprar/iphone/7-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226467,2018-05-31 18:01:23,search engine hit,898fc4e3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199204,2018-05-18 17:16:03,ad campaign hit,0f7a6eb9,/comprar/motorola/moto-x-force,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rtbhouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
759153,2018-05-19 22:23:58,viewed product,ad0b752a,NaN,8471.0,Samsung Galaxy S7,Bom,32GB,Prata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098486,2018-05-18 04:32:01,searched products,d16934ff,NaN,NaN,NaN,NaN,NaN,NaN,"12534,12548,12507,12520",Moto G5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Formateo los eventos
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'])

In [7]:
# Obtenemos un frame con solo los usuarios donde iremos mergeando los features
persons = eventos.person.to_frame().drop_duplicates()

### Región geográfica
Agregamos la región geográfica al igual que en el TP 1, para saber desde donde acceden más los distintos usuarios.

In [8]:
# Categorizamos las regiones geograficamente
eventos['geo_region'] = 0

eventos.loc[
    (eventos['region'] == 'Sao Paulo') |
    (eventos['region'] == 'Rio de Janeiro') |
    (eventos['region'] == 'Minas Gerais') |
    (eventos['region'] == 'Espirito Santo'), 
    'geo_region'
] = 'southeast'

eventos.loc[
    (eventos['region'] == 'Parana') |
    (eventos['region'] == 'Rio Grande do Sul') |
    (eventos['region'] == 'Santa Catarina'), 
    'geo_region'
] = 'south'

eventos.loc[
    (eventos['region'] == 'Federal District') |
    (eventos['region'] == 'Goias') |
    (eventos['region'] == 'Mato Grosso do Sul') |
    (eventos['region'] == 'Mato Grosso'), 
    'geo_region'
] = 'center west'

eventos.loc[
    (eventos['region'] == 'Bahia') |
    (eventos['region'] == 'Pernambuco') |
    (eventos['region'] == 'Ceara') |
    (eventos['region'] == 'Maranhao') |
    (eventos['region'] == 'Rio Grande do Norte') |
    (eventos['region'] == 'Paraíba') |
    (eventos['region'] == 'Piaui') |
    (eventos['region'] == 'Alagoas') |
    (eventos['region'] == 'Sergipe'), 
    'geo_region'
] = 'northeast'

eventos.loc[
    (eventos['region'] == 'Para') |
    (eventos['region'] == 'Amazonas') |
    (eventos['region'] == 'Tocantins') |
    (eventos['region'] == 'Amapa') |
    (eventos['region'] == 'Rondonia') |
    (eventos['region'] == 'Acre') |
    (eventos['region'] == 'Roraima'), 
    'geo_region'
] = 'north'

In [9]:
eventos.geo_region.value_counts()

0              2172865
southeast        97323
northeast        40903
south            14242
center west       9249
north             7099
Name: geo_region, dtype: int64

Creamos columnas booleanas sobre las regiones geograficas para luego agrupar por usuarios.

In [10]:
eventos['cant_accesos_southeast'] = (eventos.geo_region == 'southeast').astype(int)
eventos['cant_accesos_northeast'] = (eventos.geo_region == 'northeast').astype(int)
eventos['cant_accesos_south'] = (eventos.geo_region == 'south').astype(int)
eventos['cant_accesos_center_west'] = (eventos.geo_region == 'center west').astype(int)
eventos['cant_accesos_north'] = (eventos.geo_region == 'north').astype(int)

In [11]:
personas_y_accesos_por_region = eventos.groupby(['person']).agg({
    'cant_accesos_southeast': 'sum', 
    'cant_accesos_northeast': 'sum', 
    'cant_accesos_south': 'sum', 
    'cant_accesos_center_west': 'sum', 
    'cant_accesos_north': 'sum'
}).reset_index()
personas_y_accesos_por_region.head(5)

,person,cant_accesos_north,cant_accesos_center_west,cant_accesos_southeast,cant_accesos_south,cant_accesos_northeast
0,0008ed71,0,0,0,0,0
1,00091926,0,0,0,34,0
2,00091a7a,0,0,1,0,0
3,000ba417,0,0,6,0,0
4,000c79fe,0,0,1,0,0


Agregamos este nuevo feature a nuestro dataframe de features.

In [12]:
user_features = persons.merge(personas_y_accesos_por_region, how='inner', on='person')
user_features.head(5)

,person,cant_accesos_north,cant_accesos_center_west,cant_accesos_southeast,cant_accesos_south,cant_accesos_northeast
0,4886f805,0,0,1,0,0
1,ad93850f,0,0,5,0,0
2,0297fc1e,0,0,95,0,0
3,2d681dd8,0,0,2,0,0
4,cccea85e,0,0,22,0,0


In [13]:
personas_y_times = eventos.groupby(['person']).agg({
    'timestamp': ['min', 'max']
})
personas_y_times.reset_index(inplace=True)
personas_y_times.columns = ['person', 'time_first_event', 'time_last_event']
personas_y_times.head()

,person,time_first_event,time_last_event
0,0008ed71,2018-05-17 12:27:47,2018-05-17 16:28:37
1,00091926,2018-05-03 22:08:29,2018-05-31 19:52:03
2,00091a7a,2018-03-26 14:51:11,2018-03-26 14:56:58
3,000ba417,2018-05-17 11:11:45,2018-05-26 13:09:22
4,000c79fe,2018-05-29 00:27:47,2018-05-29 00:38:07


Agrego la cantidad de días al primer evento del año del usuario, usando como fecha para calcular las diferencias el 1 de junio de 2018 (dato del enunciado).

In [14]:
last_date = pd.to_datetime("2018/06/01")
personas_y_times['how_long_ago_was_first_event'] = (last_date - personas_y_times.time_first_event)
personas_y_times['how_long_ago_was_last_event'] = (last_date - personas_y_times.time_last_event)
personas_y_times['first_and_last_event_distance'] = (personas_y_times.time_last_event - personas_y_times.time_first_event)
personas_y_times.head()

,person,time_first_event,time_last_event,how_long_ago_was_first_event,how_long_ago_was_last_event,first_and_last_event_distance
0,0008ed71,2018-05-17 12:27:47,2018-05-17 16:28:37,14 days 11:32:13,14 days 07:31:23,0 days 04:00:50
1,00091926,2018-05-03 22:08:29,2018-05-31 19:52:03,28 days 01:51:31,0 days 04:07:57,27 days 21:43:34
2,00091a7a,2018-03-26 14:51:11,2018-03-26 14:56:58,66 days 09:08:49,66 days 09:03:02,0 days 00:05:47
3,000ba417,2018-05-17 11:11:45,2018-05-26 13:09:22,14 days 12:48:15,5 days 10:50:38,9 days 01:57:37
4,000c79fe,2018-05-29 00:27:47,2018-05-29 00:38:07,2 days 23:32:13,2 days 23:21:53,0 days 00:10:20


Queremos ver la frecuencia de eventos de los usuarios y para ello queremos saber la cantidad total de eventos que genero cada usuario


In [18]:
# Agrego cantidad total de eventos que hizo cada usuario
eventos['cant_total_eventos'] = 1
cant_total_eventos_por_usuario = eventos.groupby(['person']).agg({'cant_total_eventos': 'sum'}).reset_index()
cant_total_eventos_por_usuario.head()

,person,cant_total_eventos
0,0008ed71,6
1,00091926,448
2,00091a7a,10
3,000ba417,206
4,000c79fe,17


In [20]:
personas_times_y_eventos_totales = personas_y_times.merge(cant_total_eventos_por_usuario, how='inner', on='person')
personas_times_y_eventos_totales.head()

,person,time_first_event,time_last_event,how_long_ago_was_first_event,how_long_ago_was_last_event,first_and_last_event_distance,cant_total_eventos
0,0008ed71,2018-05-17 12:27:47,2018-05-17 16:28:37,14 days 11:32:13,14 days 07:31:23,0 days 04:00:50,6
1,00091926,2018-05-03 22:08:29,2018-05-31 19:52:03,28 days 01:51:31,0 days 04:07:57,27 days 21:43:34,448
2,00091a7a,2018-03-26 14:51:11,2018-03-26 14:56:58,66 days 09:08:49,66 days 09:03:02,0 days 00:05:47,10
3,000ba417,2018-05-17 11:11:45,2018-05-26 13:09:22,14 days 12:48:15,5 days 10:50:38,9 days 01:57:37,206
4,000c79fe,2018-05-29 00:27:47,2018-05-29 00:38:07,2 days 23:32:13,2 days 23:21:53,0 days 00:10:20,17


Pasamos la distancia entre el primer y el último evento del usuario a "días" (sumando 1 para los casos que hay 0 días de distancia). Con esto podremos dividir la cantidad de eventos totales sobre esta distancia para obtener la "cantidad promedio de eventos por día por usuario".

In [27]:
personas_times_y_eventos_totales['first_and_last_event_distance_in_days'] = personas_times_y_eventos_totales.first_and_last_event_distance.dt.days + 1
personas_times_y_eventos_totales['mean_events_per_day'] = personas_times_y_eventos_totales.cant_total_eventos / personas_times_y_eventos_totales.first_and_last_event_distance_in_days
personas_times_y_eventos_totales.head()

,person,time_first_event,time_last_event,how_long_ago_was_first_event,how_long_ago_was_last_event,first_and_last_event_distance,cant_total_eventos,first_and_last_event_distance_in_days,mean_events_per_day
0,0008ed71,2018-05-17 12:27:47,2018-05-17 16:28:37,14 days 11:32:13,14 days 07:31:23,0 days 04:00:50,6,1,6.0
1,00091926,2018-05-03 22:08:29,2018-05-31 19:52:03,28 days 01:51:31,0 days 04:07:57,27 days 21:43:34,448,28,16.0
2,00091a7a,2018-03-26 14:51:11,2018-03-26 14:56:58,66 days 09:08:49,66 days 09:03:02,0 days 00:05:47,10,1,10.0
3,000ba417,2018-05-17 11:11:45,2018-05-26 13:09:22,14 days 12:48:15,5 days 10:50:38,9 days 01:57:37,206,10,20.6
4,000c79fe,2018-05-29 00:27:47,2018-05-29 00:38:07,2 days 23:32:13,2 days 23:21:53,0 days 00:10:20,17,1,17.0


In [30]:
user_features2 = user_features.merge(personas_times_y_eventos_totales, how='inner', on='person')
# user_features2.to_csv('features_ariel.csv')